In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score


from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import PredefinedSplit

from imblearn.over_sampling import ADASYN, RandomOverSampler, SMOTE
from imblearn import FunctionSampler  # to use a idendity sampler

from pandas.api.types import CategoricalDtype

In [2]:
def confusion_matrix_relaxed(confusion_matrix_temp):
    # returns the relative number of predictions which are
    # +/- 1 compared to the actual result should be provided.
    i_cnt = confusion_matrix_temp.shape[0]
    j_cnt = confusion_matrix_temp.shape[1]
    bin_matrix_temp = confusion_matrix_temp.copy()
    for i in range(0, i_cnt):
        for j in range(0, j_cnt):
            if (abs(i-j)==1):
                bin_matrix_temp[i,j] = abs(i-j)
            else:
                bin_matrix_temp[i,j] = 0
    return ((confusion_matrix_temp*bin_matrix_temp).sum()) / (confusion_matrix_temp.sum())

In [3]:
def confusion_matrix_perc(confusion_matrix_temp):
    # returns the relative number of predictions which are
    # on main diagonal compared to the actual result should be provided.
    i_cnt = confusion_matrix_temp.shape[0]
    j_cnt = confusion_matrix_temp.shape[1]
    return ((confusion_matrix_temp*np.eye(i_cnt)).sum()) / (confusion_matrix_temp.sum())

In [4]:
def write_scoring(model, test_X, test_y, RMSE=True):
### Write several scores to the output ###
    print(model.estimator.named_steps)
    print('Accuracy Score: ', sklearn.metrics.accuracy_score(test_y, model.predict(test_X)))
    print('Balanced Accuracy Score: ', sklearn.metrics.balanced_accuracy_score(test_y, model.predict(test_X)))
    print('Cohen-Kappa-Score: ', sklearn.metrics.cohen_kappa_score(test_y, model.predict(test_X)))
    print('F1-Score: ', sklearn.metrics.f1_score(test_y, model.predict(test_X), average='weighted'))
    if RMSE == True:
        print('RMSE: ', np.sqrt(sklearn.metrics.mean_squared_error(test_y, model.predict(test_X))))
        #print('ROC-AUC-Score: ', sklearn.metrics.roc_auc_score(test_y, 
        #                                                       model.predict(test_X), 
        #                                                       average='weighted', 
        #                                                       multi_class='ovo'))
    print('__________________________________\n')
    print('Confusion matrix: \n', sklearn.metrics.confusion_matrix(test_y, model.predict(test_X)))
    print(confusion_matrix_relaxed(sklearn.metrics.confusion_matrix(test_y, model.predict(test_X))))    
    print(confusion_matrix_perc(sklearn.metrics.confusion_matrix(test_y, model.predict(test_X))))
    print(sklearn.metrics.classification_report(test_y, model.predict(test_X)))

In [5]:
SUFFIX = '_MIN_FOUR_VARIABLES_DATASET_NR_'#'_recategorized'
FILE_TO_LOAD = 'CHES2019_experts.csv' # 'CHES2019_experts_recategorized_4l3c4r.csv' #' CHES2019_experts_recategorized.csv'#'CHES2019_experts.csv', 
FOLDER_PREFIX = '' #'base_data/' #'recategorized_4l3c4r/' # 
FOLDER_IMPUTE_PREFIX = 'imputed_data/'

In [6]:
prefix = 'data_lrgen_base_'
X_train_lrgen_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(0) + '.csv', index_col=0)
y_train_lrgen_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(0) + '.csv', index_col=0)
X_valid_lrgen_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(0) + '.csv', index_col=0)
y_valid_lrgen_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(0) + '.csv', index_col=0)
X_test_lrgen_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(0) + '.csv', index_col=0)
y_test_lrgen_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(0) + '.csv', index_col=0)

X_train_lrgen_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(1) + '.csv', index_col=0)
y_train_lrgen_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(1) + '.csv', index_col=0)
X_valid_lrgen_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(1) + '.csv', index_col=0)
y_valid_lrgen_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(1) + '.csv', index_col=0)
X_test_lrgen_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(1) + '.csv', index_col=0)
y_test_lrgen_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(1) + '.csv', index_col=0)

X_train_lrgen_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(2) + '.csv', index_col=0)
y_train_lrgen_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(2) + '.csv', index_col=0)
X_valid_lrgen_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(2) + '.csv', index_col=0)
y_valid_lrgen_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(2) + '.csv', index_col=0)
X_test_lrgen_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(2) + '.csv', index_col=0)
y_test_lrgen_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(2) + '.csv', index_col=0)

X_train_lrgen_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(3) + '.csv', index_col=0)
y_train_lrgen_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(3) + '.csv', index_col=0)
X_valid_lrgen_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(3) + '.csv', index_col=0)
y_valid_lrgen_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(3) + '.csv', index_col=0)
X_test_lrgen_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(3) + '.csv', index_col=0)
y_test_lrgen_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(3) + '.csv', index_col=0)

X_train_lrgen_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(4) + '.csv', index_col=0)
y_train_lrgen_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(4) + '.csv', index_col=0)
X_valid_lrgen_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(4) + '.csv', index_col=0)
y_valid_lrgen_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(4) + '.csv', index_col=0)
X_test_lrgen_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(4) + '.csv', index_col=0)
y_test_lrgen_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(4) + '.csv', index_col=0)

In [7]:
prefix = 'data_lrecon_base_'
X_train_lrecon_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(0) + '.csv', index_col=0)
y_train_lrecon_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(0) + '.csv', index_col=0)
X_valid_lrecon_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(0) + '.csv', index_col=0)
y_valid_lrecon_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(0) + '.csv', index_col=0)
X_test_lrecon_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(0) + '.csv', index_col=0)
y_test_lrecon_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(0) + '.csv', index_col=0)

X_train_lrecon_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(1) + '.csv', index_col=0)
y_train_lrecon_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(1) + '.csv', index_col=0)
X_valid_lrecon_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(1) + '.csv', index_col=0)
y_valid_lrecon_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(1) + '.csv', index_col=0)
X_test_lrecon_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(1) + '.csv', index_col=0)
y_test_lrecon_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(1) + '.csv', index_col=0)

X_train_lrecon_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(2) + '.csv', index_col=0)
y_train_lrecon_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(2) + '.csv', index_col=0)
X_valid_lrecon_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(2) + '.csv', index_col=0)
y_valid_lrecon_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(2) + '.csv', index_col=0)
X_test_lrecon_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(2) + '.csv', index_col=0)
y_test_lrecon_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(2) + '.csv', index_col=0)

X_train_lrecon_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(3) + '.csv', index_col=0)
y_train_lrecon_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(3) + '.csv', index_col=0)
X_valid_lrecon_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(3) + '.csv', index_col=0)
y_valid_lrecon_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(3) + '.csv', index_col=0)
X_test_lrecon_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(3) + '.csv', index_col=0)
y_test_lrecon_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(3) + '.csv', index_col=0)

X_train_lrecon_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(4) + '.csv', index_col=0)
y_train_lrecon_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(4) + '.csv', index_col=0)
X_valid_lrecon_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(4) + '.csv', index_col=0)
y_valid_lrecon_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(4) + '.csv', index_col=0)
X_test_lrecon_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(4) + '.csv', index_col=0)
y_test_lrecon_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(4) + '.csv', index_col=0)

In [8]:
prefix = 'data_galtan_base_'
X_train_galtan_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(0) + '.csv', index_col=0)
y_train_galtan_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(0) + '.csv', index_col=0)
X_valid_galtan_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(0) + '.csv', index_col=0)
y_valid_galtan_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(0) + '.csv', index_col=0)
X_test_galtan_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(0) + '.csv', index_col=0)
y_test_galtan_0 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(0) + '.csv', index_col=0)

X_train_galtan_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(1) + '.csv', index_col=0)
y_train_galtan_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(1) + '.csv', index_col=0)
X_valid_galtan_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(1) + '.csv', index_col=0)
y_valid_galtan_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(1) + '.csv', index_col=0)
X_test_galtan_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(1) + '.csv', index_col=0)
y_test_galtan_1 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(1) + '.csv', index_col=0)

X_train_galtan_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(2) + '.csv', index_col=0)
y_train_galtan_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(2) + '.csv', index_col=0)
X_valid_galtan_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(2) + '.csv', index_col=0)
y_valid_galtan_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(2) + '.csv', index_col=0)
X_test_galtan_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(2) + '.csv', index_col=0)
y_test_galtan_2 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(2) + '.csv', index_col=0)

X_train_galtan_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(3) + '.csv', index_col=0)
y_train_galtan_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(3) + '.csv', index_col=0)
X_valid_galtan_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(3) + '.csv', index_col=0)
y_valid_galtan_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(3) + '.csv', index_col=0)
X_test_galtan_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(3) + '.csv', index_col=0)
y_test_galtan_3 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(3) + '.csv', index_col=0)

X_train_galtan_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(4) + '.csv', index_col=0)
y_train_galtan_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(4) + '.csv', index_col=0)
X_valid_galtan_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(4) + '.csv', index_col=0)
y_valid_galtan_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(4) + '.csv', index_col=0)
X_test_galtan_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(4) + '.csv', index_col=0)
y_test_galtan_4 = pd.read_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(4) + '.csv', index_col=0)

# Train model with CV

In [9]:
### oversampling required?

In [10]:
########### PARAMETERS 
scoring_method = 'balanced_accuracy'#['balanced_accuracy', 'f1_samples', 'roc_auc_ovo_weighted']
cv_nr = 5

In [11]:
### create related data collection for iteration
data_temp = [[X_train_lrgen_0, X_valid_lrgen_0,y_train_lrgen_0, y_valid_lrgen_0, X_test_lrgen_0, y_test_lrgen_0],
             [X_train_lrgen_1, X_valid_lrgen_1,y_train_lrgen_1, y_valid_lrgen_1, X_test_lrgen_1, y_test_lrgen_1],
             [X_train_lrgen_2, X_valid_lrgen_2,y_train_lrgen_2, y_valid_lrgen_2, X_test_lrgen_2, y_test_lrgen_2],
             [X_train_lrgen_3, X_valid_lrgen_3,y_train_lrgen_3, y_valid_lrgen_3, X_test_lrgen_3, y_test_lrgen_3],
             [X_train_lrgen_4, X_valid_lrgen_4,y_train_lrgen_4, y_valid_lrgen_4, X_test_lrgen_4, y_test_lrgen_4]]

## SVC

In [12]:
# Create the parameter grid
param_grid_svc = [{
                    'svc__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'svc__kernel': ['linear'],
                    'svc__degree': [1,2,3,4,5]
                  },
                 {
                    'svc__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'svc__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'svc__kernel': ['rbf']
                 }]



In [13]:
### prepare data
#X_train = pd.concat([X_train_lrgen_0, X_valid_lrgen_0])
#y_train = pd.concat([y_train_lrgen_0, y_valid_lrgen_0])
#test_fold = np.full(len(X_train), -1)
#test_fold[-len(X_valid_lrgen_0):] = 1
#ps = PredefinedSplit(test_fold)

In [14]:
# Create pipeline
pipelineSVC = make_pipeline(StandardScaler(), SVC(random_state=1))

for i in range(0,5):
    X_train = pd.concat([data_temp[i][0], data_temp[i][1]])
    y_train = pd.concat([data_temp[i][2], data_temp[i][3]])
    test_fold = np.full(len(X_train), -1)
    test_fold[-len(data_temp[i][1]):] = 1
    ps = PredefinedSplit(test_fold)
    #print(ps.get_n_splits())
    # Create an instance of GridSearch Cross-validation estimator
    gsSVC = GridSearchCV(estimator=pipelineSVC,
                         param_grid = param_grid_svc,
                         scoring=scoring_method,
                         cv=ps,
                         refit=True,
                         n_jobs=1)
    # Train the SVM classifier
    gsSVC.fit(X_train, np.ravel(y_train))
    print('SVC dataset number ' + str(i))
    # Print the training score of the best model
    print(gsSVC.best_score_)
    # Print the model parameters of the best model
    print(gsSVC.best_params_)
    # Print the model score on the test data using GridSearchCV score method
    #print('Test accuracy: %.3f' % gsSVC.score(data_temp[i][4], data_temp[i][5]))
    clfSVC = gsSVC.best_estimator_
    print('Test accuracy: %.3f' % clfSVC.score(data_temp[i][4], data_temp[i][5]))
    # Print the model score on the test data using Best estimator instance
    #clfSVC = gsSVC.best_estimator_
    print(clfSVC)
    write_scoring(gsSVC, data_temp[i][4], data_temp[i][5], RMSE=True)
    del(X_train, y_train, test_fold, ps, gsSVC, clfSVC)
    
    #print('Test accuracy: %.3f' % clfSVC.score(X_test, y_test))


SVC dataset number 0
0.40286238850893774
{'svc__C': 10.0, 'svc__gamma': 0.05, 'svc__kernel': 'rbf'}
Test accuracy: 0.360
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=10.0, gamma=0.05, random_state=1))])
{'standardscaler': StandardScaler(), 'svc': SVC(random_state=1)}
Accuracy Score:  0.35988200589970504
Balanced Accuracy Score:  0.35613139992137377
Cohen-Kappa-Score:  0.2852965179542982
F1-Score:  0.35548949541947106
RMSE:  1.4644257484971224
__________________________________

Confusion matrix: 
 [[ 1  5  1  1  0  0  0  0  0  0  0]
 [ 0 14  5  3  1  0  0  0  0  1  0]
 [ 0  3 14  8  2  1  0  0  1  1  0]
 [ 0  2  7 11  8  1  2  0  1  0  1]
 [ 0  0  3  5 14 10  5  1  0  0  0]
 [ 0  1  2  1 12 17  7  6  0  0  1]
 [ 0  0  0  0  5 11  6 11  6  0  0]
 [ 0  0  0  0  5  5  9 18  5  3  1]
 [ 0  0  0  0  0  0  0 14 13  6  3]
 [ 0  0  0  0  0  0  0  1  7  6  8]
 [ 0  0  0  0  0  0  0  0  0  8  8]]
0.43952802359882004
0.35988200589970504
              precisi

## Random Forest Classification

In [15]:
pipelineRFC = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=1, criterion='entropy')) 
# Create the parameter grid
                            
param_grid_rfc = [{
    'randomforestclassifier__max_depth':[2, 3, 4, 5, 6, 7, 8, 9, 10],
    'randomforestclassifier__max_features':[2, 3, 4, 5, 6],
    'randomforestclassifier__n_estimators':range(5, 105, 5)#,
    #'randomforestclassifier__criterion:':['gini', 'entropy']
}]


In [16]:
for i in range(0,5):
    X_train = pd.concat([data_temp[i][0], data_temp[i][1]])
    y_train = pd.concat([data_temp[i][2], data_temp[i][3]])
    test_fold = np.full(len(X_train), -1)
    test_fold[-len(data_temp[i][1]):] = 1
    ps = PredefinedSplit(test_fold)
    #print(ps.get_n_splits())
    # Create an instance of GridSearch Cross-validation estimator
    gsRFC = GridSearchCV(estimator=pipelineRFC,
                         param_grid = param_grid_rfc,
                         scoring=scoring_method,
                         cv=cv_nr,
                         refit=True,
                         n_jobs=1)
    # Train the RandomForestClassifier
    gsRFC = gsRFC.fit(X_train, np.ravel(y_train))
    print('RFC dataset number ' + str(i))
    # Print the training score of the best model
    print(gsRFC.best_score_)
    # Print the model parameters of the best model
    print(gsRFC.best_params_)
    # Print the model score on the test data using GridSearchCV score method
    clfRFC = gsRFC.best_estimator_
    print('Test accuracy: %.3f' % clfRFC.score(data_temp[i][4], data_temp[i][5]))
    # Print the model score on the test data using Best estimator instance
    print(clfRFC)
    write_scoring(gsRFC, data_temp[i][4], data_temp[i][5], RMSE=True)
    del(X_train, y_train, test_fold, ps, gsRFC, clfRFC)
    
    #print('Test accuracy: %.3f' % clfSVC.score(X_test, y_test))

RFC dataset number 0
0.37230959028326327
{'randomforestclassifier__max_depth': 9, 'randomforestclassifier__max_features': 2, 'randomforestclassifier__n_estimators': 60}
Test accuracy: 0.330
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=9,
                                        max_features=2, n_estimators=60,
                                        random_state=1))])
{'standardscaler': StandardScaler(), 'randomforestclassifier': RandomForestClassifier(criterion='entropy', random_state=1)}
Accuracy Score:  0.3303834808259587
Balanced Accuracy Score:  0.33607164250048743
Cohen-Kappa-Score:  0.25254968238242337
F1-Score:  0.32923509232166487
RMSE:  1.6411018674150117
__________________________________

Confusion matrix: 
 [[ 2  3  2  0  0  1  0  0  0  0  0]
 [ 3  9  6  2  3  0  0  0  0  0  1]
 [ 0  5 13  6  1  3  1  0  0  0  1]
 [ 1  0  9 10  7  1  2  2  1  0  0]
 [ 0

RFC dataset number 4
0.383902687827189
{'randomforestclassifier__max_depth': 9, 'randomforestclassifier__max_features': 2, 'randomforestclassifier__n_estimators': 40}
Test accuracy: 0.345
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=9,
                                        max_features=2, n_estimators=40,
                                        random_state=1))])
{'standardscaler': StandardScaler(), 'randomforestclassifier': RandomForestClassifier(criterion='entropy', random_state=1)}
Accuracy Score:  0.34513274336283184
Balanced Accuracy Score:  0.355769998621134
Cohen-Kappa-Score:  0.26833109724085624
F1-Score:  0.34167380391664387
RMSE:  1.687189280909679
__________________________________

Confusion matrix: 
 [[ 3  3  0  0  0  0  0  0  0  0  0]
 [ 0  7  8  2  1  0  0  0  0  0  0]
 [ 0  5 12  4  6  1  1  0  0  0  0]
 [ 1  1  5  7 12  1  1  1  0  0  1]
 [ 0  1 

## Logistic Regression Classification

In [17]:
pipelineLR = make_pipeline(StandardScaler(), LogisticRegression(random_state=1, penalty='l2', solver='lbfgs', max_iter=1000))
# Create the parameter grid
param_grid_lr = [{
    'logisticregression__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
}]

In [18]:
for i in range(0,5):
    X_train = pd.concat([data_temp[i][0], data_temp[i][1]])
    y_train = pd.concat([data_temp[i][2], data_temp[i][3]])
    test_fold = np.full(len(X_train), -1)
    test_fold[-len(data_temp[i][1]):] = 1
    ps = PredefinedSplit(test_fold)
    #print(ps.get_n_splits())
    # Create an instance of GridSearch Cross-validation estimator
    gsLR = GridSearchCV(estimator=pipelineLR,
                         param_grid = param_grid_lr,
                         scoring=scoring_method,
                         cv=cv_nr,
                         refit=True,
                         n_jobs=1)
    # Train the LogisticRegression Classifier
    gsLR = gsLR.fit(X_train, np.ravel(y_train))
    # Print the training score of the best model
    print(gsLR.best_score_)
    # Print the model parameters of the best model
    print(gsLR.best_params_)
    # Print the test score of the best model
    clfLR = gsLR.best_estimator_
    print('Test accuracy: %.3f' % clfLR.score(data_temp[i][4], data_temp[i][5]))
    # Print the model score on the test data using Best estimator instance
    print(clfLR)
    write_scoring(gsLR, data_temp[i][4], data_temp[i][5], RMSE=True)
    del(X_train, y_train, test_fold, ps, gsLR, clfLR)
    




0.3467638478521728
{'logisticregression__C': 10.0}
Test accuracy: 0.360
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(C=10.0, max_iter=1000, random_state=1))])
{'standardscaler': StandardScaler(), 'logisticregression': LogisticRegression(max_iter=1000, random_state=1)}
Accuracy Score:  0.35988200589970504
Balanced Accuracy Score:  0.37032177022057683
Cohen-Kappa-Score:  0.2856019111991609
F1-Score:  0.34250418732048743
RMSE:  1.5514766018273365
__________________________________

Confusion matrix: 
 [[ 1  3  3  1  0  0  0  0  0  0  0]
 [ 1  9  9  1  3  0  0  0  0  1  0]
 [ 0  2 19  3  4  0  0  1  0  0  1]
 [ 0  1 10  8  8  4  1  0  0  1  0]
 [ 0  0  3  5 12 13  2  3  0  0  0]
 [ 0  2  1  3 10 20  4  4  2  0  1]
 [ 0  1  0  0  3 12  5 12  5  1  0]
 [ 0  0  0  0  2  8  7 16  8  3  2]
 [ 0  0  0  0  3  1  0 10 13  6  3]
 [ 0  0  0  0  0  0  1  2  3  4 12]
 [ 0  0  0  0  0  0  0  0  0  1 15]]
0.4100294985250

## ADABoost Classification (Ensemble Method)

In [19]:
pipelineADAB = make_pipeline(StandardScaler(), AdaBoostClassifier(random_state=1))
# Create the parameter grid
param_grid_ADAB = [{
    'adaboostclassifier__n_estimators': [2, 10, 20, 30, 40, 50, 100]
}]

In [21]:
for i in range(0,5):
    X_train = pd.concat([data_temp[i][0], data_temp[i][1]])
    y_train = pd.concat([data_temp[i][2], data_temp[i][3]])
    test_fold = np.full(len(X_train), -1)
    test_fold[-len(data_temp[i][1]):] = 1
    ps = PredefinedSplit(test_fold)
    # Create an instance of GridSearch Cross-validation estimator
    gsADAB = GridSearchCV(estimator=pipelineADAB,
                         param_grid = param_grid_ADAB,
                         scoring=scoring_method,
                         cv=cv_nr,
                         refit=True,
                         n_jobs=1)
    # Train the LogisticRegression Classifier
    gsADAB = gsADAB.fit(X_train, np.ravel(y_train))
    # Print the training score of the best model
    print(gsADAB.best_score_)
    # Print the model parameters of the best model
    print(gsADAB.best_params_)
    # Print the test score of the best model
    clfADAB = gsADAB.best_estimator_
    print('Test accuracy: %.3f' % clfADAB.score(data_temp[i][4], data_temp[i][5]))
    # Print the model score on the test data using Best estimator instance
    print(clfADAB)
    write_scoring(gsADAB, data_temp[i][4], data_temp[i][5], RMSE=True)
    del(X_train, y_train, test_fold, ps, gsADAB)

0.31158810780664864
{'adaboostclassifier__n_estimators': 20}
Test accuracy: 0.242
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('adaboostclassifier',
                 AdaBoostClassifier(n_estimators=20, random_state=1))])
{'standardscaler': StandardScaler(), 'adaboostclassifier': AdaBoostClassifier(random_state=1)}
Accuracy Score:  0.24188790560471976
Balanced Accuracy Score:  0.3017308723274978
Cohen-Kappa-Score:  0.165616380631321
F1-Score:  0.20520196027124787
RMSE:  1.7599275930399418
__________________________________

Confusion matrix: 
 [[ 6  0  0  2  0  0  0  0  0  0  0]
 [14  1  1  7  0  0  0  0  0  0  1]
 [14  0  1 12  0  1  0  1  0  1  0]
 [ 5  0  1 16  3  4  3  0  0  0  1]
 [ 0  0  1 13  8  7  8  0  0  1  0]
 [ 1  0  1  6 10  9 18  0  0  1  1]
 [ 0  0  0  4  2 10 17  4  0  2  0]
 [ 0  0  1  1  5  5 23  3  1  5  2]
 [ 0  1  0  0  0  0 10  3  3 14  5]
 [ 0  0  0  0  0  1  0  1  2  6 12]
 [ 0  0  0  0  0  0  0  0  0  4 12]]
0.44837758112094395
0.241887

/home/gernot/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gernot/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gernot/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3031208361836166
{'adaboostclassifier__n_estimators': 10}
Test accuracy: 0.289
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('adaboostclassifier',
                 AdaBoostClassifier(n_estimators=10, random_state=1))])
{'standardscaler': StandardScaler(), 'adaboostclassifier': AdaBoostClassifier(random_state=1)}
Accuracy Score:  0.2890855457227139
Balanced Accuracy Score:  0.2642030583457954
Cohen-Kappa-Score:  0.2061429931787706
F1-Score:  0.25899153533098823
RMSE:  1.848223255870303
__________________________________

Confusion matrix: 
 [[ 0  0  0  6  0  0  0  0  0  0  0]
 [ 0  0  5 13  0  0  0  0  0  0  0]
 [ 0  0 13  9  4  1  1  0  0  1  0]
 [ 0  0 12  2 12  1  2  0  0  0  1]
 [ 0  2 10  3 20  2  9  0  0  0  3]
 [ 0  0  8  0  9  5 18  0  1  0  1]
 [ 0  0  4  1  2  2 25  2  1  2  0]
 [ 0  1  0  0  4  2 22  2  2  7  0]
 [ 0  0  2  0  5  0 11  1 11 11  0]
 [ 0  0  0  0  0  0  4  1  5  8  6]
 [ 0  0  0  0  0  0  0  0  2  7 12]]
0.3775811209439528
0.289085545

/home/gernot/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gernot/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gernot/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
